# for squats

In [12]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas as pd
from scipy.spatial.distance import cosine
import os

class ExerciseRecognizer:
    def __init__(self, master_video_path, target_reps, landmarks_csv="squat_mas.csv"):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.mp_drawing = mp.solutions.drawing_utils
        self.master_video_path = master_video_path
        self.landmarks_csv = landmarks_csv
        self.master_angles = self.load_or_extract_master_angles()
        self.rep_count = 0
        self.target_reps = target_reps
        self.exercise_started = False
        self.previous_angle = None
        self.final_similarity = 0
        self.start_time = None
        self.end_time = None
    
    def get_landmarks(self, frame):
        """Extract pose landmarks from a frame."""
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(frame_rgb)
        if results.pose_landmarks:
            return results.pose_landmarks
        return None
    
    def calculate_angle(self, point1, point2, point3):
        """Calculate the angle between three keypoints."""
        a = np.array(point1)
        b = np.array(point2)
        c = np.array(point3)
        
        ba = a - b
        bc = c - b
        
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
        return np.degrees(angle)
    
    def compare_with_master(self, user_angles):
        """Compare input squat angles with master video angles."""
        if len(self.master_angles) == 0 or len(user_angles) == 0:
            return 0.0
        min_length = min(len(self.master_angles), len(user_angles))
        master_sampled = self.master_angles[:min_length]
        user_sampled = user_angles[:min_length]
        similarity = 1 - cosine(master_sampled, user_sampled)
        return max(0, similarity * 100)
    
    def load_or_extract_master_angles(self):
        """Load master angles from CSV or extract them from the video."""
        if os.path.exists(self.landmarks_csv):
            print("Loading cached master angles from CSV...")
            df = pd.read_csv(self.landmarks_csv)
            return df["angles"].values if "angles" in df.columns else []
        else:
            print("Extracting master angles from video and saving to CSV...")
            angles = self.extract_master_angles()
            df = pd.DataFrame({"angles": angles})
            df.to_csv(self.landmarks_csv, index=False)
            return angles
    
    def extract_master_angles(self):
        """Extract squat angles from master video."""
        cap = cv2.VideoCapture(self.master_video_path)
        angles = []
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            landmarks = self.get_landmarks(frame)
            if landmarks:
                lm = landmarks.landmark
                left_hip = [lm[self.mp_pose.PoseLandmark.LEFT_HIP.value].x, lm[self.mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [lm[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x, lm[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [lm[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x, lm[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                angles.append(self.calculate_angle(left_hip, left_knee, left_ankle))
        cap.release()
        return angles if angles else [0]
    
    def process_webcam(self):
        cap = cv2.VideoCapture(0)
        user_angles = []
        self.start_time = time.time()
        
        while cap.isOpened() and self.rep_count < self.target_reps:
            ret, frame = cap.read()
            if not ret:
                break
            
            landmarks = self.get_landmarks(frame)
            if landmarks:
                lm = landmarks.landmark
                left_hip = [lm[self.mp_pose.PoseLandmark.LEFT_HIP.value].x, lm[self.mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [lm[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x, lm[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [lm[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x, lm[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                angle = self.calculate_angle(left_hip, left_knee, left_ankle)
                user_angles.append(angle)
                
                if self.previous_angle is not None:
                    if angle < 90 and not self.exercise_started:
                        self.exercise_started = True
                    elif angle > 160 and self.exercise_started:
                        self.rep_count += 1
                        self.exercise_started = False
                self.previous_angle = angle
                
                self.final_similarity = self.compare_with_master(user_angles)
                self.mp_drawing.draw_landmarks(frame, landmarks, self.mp_pose.POSE_CONNECTIONS)
                cv2.putText(frame, f'Similarity: {int(self.final_similarity)}%', (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(frame, f'Reps: {self.rep_count}/{self.target_reps}', (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            cv2.imshow('Exercise Analysis', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
        self.end_time = time.time()
        cap.release()
        cv2.destroyAllWindows()
        total_time = self.end_time - self.start_time
        print(f'Final Similarity Score: {int(self.final_similarity)}%')
        print(f'Total Time Taken: {total_time:.2f} seconds')
        print(f'Total Reps Completed: {self.rep_count}/{self.target_reps}')

if __name__ == "__main__":
    master_video = r"C:/AIML/assets_master/squats_master.mp4"
    target_reps = 5
    recognizer = ExerciseRecognizer(master_video, target_reps)
    recognizer.process_webcam()


Loading cached master angles from CSV...
Final Similarity Score: 87%
Total Time Taken: 27.65 seconds
Total Reps Completed: 5/5


# for pushups

In [19]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas as pd
from scipy.spatial.distance import cosine
import os

class PushUpRecognizer:
    def __init__(self, master_video_path, target_reps, landmarks_csv="pushup_mas.csv"):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.mp_drawing = mp.solutions.drawing_utils
        self.master_video_path = master_video_path
        self.landmarks_csv = landmarks_csv
        self.master_landmarks = self.load_or_extract_master_landmarks()
        self.rep_count = 0
        self.target_reps = target_reps
        self.exercise_started = False
        self.previous_angle = None
        self.final_similarity = 0
        self.start_time = None
        self.end_time = None
        self.user_landmarks_list = []

    def get_landmarks(self, frame):
        """Extract pose landmarks from a frame."""
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(frame_rgb)
        if results.pose_landmarks:
            return [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark], results.pose_landmarks
        return None, None
    
    def calculate_similarity(self):
        """Compute cosine similarity between master and user pose landmarks."""
        if len(self.master_landmarks) == 0 or len(self.user_landmarks_list) == 0:
            return 0.0
        
        min_length = min(len(self.master_landmarks), len(self.user_landmarks_list))
        master_sampled = np.array(self.master_landmarks[:min_length]).flatten()
        user_sampled = np.array(self.user_landmarks_list[:min_length]).flatten()
        
        similarity = 1 - cosine(master_sampled, user_sampled)
        return max(0, similarity * 100)
    
    def load_or_extract_master_landmarks(self):
        """Load master landmarks from CSV or extract them from the video."""
        if os.path.exists(self.landmarks_csv):
            print("Loading cached master landmarks from CSV...")
            df = pd.read_csv(self.landmarks_csv)
            return df.values.tolist()
        else:
            print("Extracting master landmarks from video and saving to CSV...")
            landmarks = self.extract_master_landmarks()
            df = pd.DataFrame(landmarks)
            df.to_csv(self.landmarks_csv, index=False)
            return landmarks
    
    def extract_master_landmarks(self):
        """Extract pose landmarks from master video."""
        cap = cv2.VideoCapture(self.master_video_path)
        master_landmarks = []
        
        if not cap.isOpened():
            print("Error: Unable to open master video.")
            return []
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            landmarks, _ = self.get_landmarks(frame)
            if landmarks:
                master_landmarks.append(np.array(landmarks).flatten().tolist())
        cap.release()
        
        return master_landmarks if len(master_landmarks) > 1 else []
    
    def process_webcam(self):
        cap = cv2.VideoCapture(0)
        self.start_time = time.time()
        
        while cap.isOpened() and self.rep_count < self.target_reps:
            ret, frame = cap.read()
            if not ret:
                break
            
            landmarks, pose_landmarks = self.get_landmarks(frame)
            if landmarks:
                self.user_landmarks_list = [np.array(landmarks).flatten().tolist()]
                
                if len(self.master_landmarks) > 0:
                    self.final_similarity = self.calculate_similarity()
                
                # Counting reps based on elbow angle
                left_shoulder = landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                left_elbow = landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW.value]
                left_wrist = landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value]
                
                angle = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
                
                if self.previous_angle is not None:
                    if angle < 90 and not self.exercise_started:
                        self.exercise_started = True
                    elif angle > 160 and self.exercise_started:
                        self.rep_count += 1
                        self.exercise_started = False
                self.previous_angle = angle
                
                self.mp_drawing.draw_landmarks(frame, pose_landmarks, self.mp_pose.POSE_CONNECTIONS,
                                               self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                               self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2))
                
                cv2.putText(frame, f'Similarity: {int(self.final_similarity)}%', (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(frame, f'Reps: {self.rep_count}/{self.target_reps}', (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            cv2.imshow('Push-Up Analysis', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
        self.end_time = time.time()
        cap.release()
        cv2.destroyAllWindows()
        total_time = self.end_time - self.start_time
        print(f'Final Form Similarity: {int(self.final_similarity)}%')
        print(f'Total Time Taken: {total_time:.2f} seconds')
        print(f'Total Reps Completed: {self.rep_count}/{self.target_reps}')
    
    def calculate_angle(self, point1, point2, point3):
        """Calculate the angle between three keypoints."""
        a = np.array(point1[:2])
        b = np.array(point2[:2])
        c = np.array(point3[:2])
        
        ba = a - b
        bc = c - b
        
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
        return np.degrees(angle)

if __name__ == "__main__":
    master_video = r"C:\AIML\assets_master\pushup_master.mp4"
    target_reps = 10
    recognizer = PushUpRecognizer(master_video, target_reps)
    recognizer.process_webcam()

Extracting master landmarks from video and saving to CSV...
Final Form Similarity: 87%
Total Time Taken: 20.45 seconds
Total Reps Completed: 10/10


# for crunches 

In [22]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas as pd
from scipy.spatial.distance import cosine
import os

class CrunchRecognizer:
    def __init__(self, master_video_path, target_reps, landmarks_csv="crunch_mas.csv"):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.mp_drawing = mp.solutions.drawing_utils
        self.master_video_path = master_video_path
        self.landmarks_csv = landmarks_csv
        self.master_landmarks = self.load_or_extract_master_landmarks()
        self.rep_count = 0
        self.target_reps = target_reps
        self.exercise_started = False
        self.previous_angle = None
        self.final_similarity = 0
        self.start_time = None
        self.end_time = None
        self.user_landmarks_list = []

    def get_landmarks(self, frame):
        """Extract pose landmarks from a frame."""
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(frame_rgb)
        if results.pose_landmarks:
            return [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark], results.pose_landmarks
        return None, None
    
    def calculate_similarity(self):
        """Compute cosine similarity between master and user pose landmarks."""
        if len(self.master_landmarks) == 0 or len(self.user_landmarks_list) == 0:
            return 0.0
        
        min_length = min(len(self.master_landmarks), len(self.user_landmarks_list))
        master_sampled = np.array(self.master_landmarks[:min_length]).flatten()
        user_sampled = np.array(self.user_landmarks_list[:min_length]).flatten()
        
        similarity = 1 - cosine(master_sampled, user_sampled)
        return max(0, similarity * 100)
    
    def load_or_extract_master_landmarks(self):
        """Load master landmarks from CSV or extract them from the video."""
        if os.path.exists(self.landmarks_csv):
            print("Loading cached master landmarks from CSV...")
            df = pd.read_csv(self.landmarks_csv)
            return df.values.tolist()
        else:
            print("Extracting master landmarks from video and saving to CSV...")
            landmarks = self.extract_master_landmarks()
            df = pd.DataFrame(landmarks)
            df.to_csv(self.landmarks_csv, index=False)
            return landmarks
    
    def extract_master_landmarks(self):
        """Extract pose landmarks from master video."""
        cap = cv2.VideoCapture(self.master_video_path)
        master_landmarks = []
        
        if not cap.isOpened():
            print("Error: Unable to open master video.")
            return []
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            landmarks, _ = self.get_landmarks(frame)
            if landmarks:
                master_landmarks.append(np.array(landmarks).flatten().tolist())
        cap.release()
        
        return master_landmarks if len(master_landmarks) > 1 else []
    
    def calculate_angle(self, point1, point2, point3):
        """Calculate the angle between three keypoints."""
        a = np.array(point1[:2])
        b = np.array(point2[:2])
        c = np.array(point3[:2])
        
        ba = a - b
        bc = c - b
        
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
        return np.degrees(angle)
    
    def process_webcam(self):
        cap = cv2.VideoCapture(0)
        self.start_time = time.time()
        
        while cap.isOpened() and self.rep_count < self.target_reps:
            ret, frame = cap.read()
            if not ret:
                break
            
            landmarks, pose_landmarks = self.get_landmarks(frame)
            if landmarks:
                self.user_landmarks_list = [np.array(landmarks).flatten().tolist()]
                
                if len(self.master_landmarks) > 0:
                    self.final_similarity = self.calculate_similarity()
                
                # Counting reps based on hip movement
                left_hip = landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value]
                left_shoulder = landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                left_knee = landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value]
                
                angle = self.calculate_angle(left_shoulder, left_hip, left_knee)
                
                if self.previous_angle is not None:
                    if angle < 45 and not self.exercise_started:
                        self.exercise_started = True
                    elif angle > 100 and self.exercise_started:
                        self.rep_count += 1
                        self.exercise_started = False
                self.previous_angle = angle
                
                self.mp_drawing.draw_landmarks(frame, pose_landmarks, self.mp_pose.POSE_CONNECTIONS,
                                               self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                               self.mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2))
                
                cv2.putText(frame, f'Similarity: {int(self.final_similarity)}%', (50, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.putText(frame, f'Reps: {self.rep_count}/{self.target_reps}', (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            cv2.imshow('Crunch Analysis', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
        self.end_time = time.time()
        cap.release()
        cv2.destroyAllWindows()
        total_time = self.end_time - self.start_time
        print(f'Final Form Similarity: {int(self.final_similarity)}%')
        print(f'Total Time Taken: {total_time:.2f} seconds')
        print(f'Total Reps Completed: {self.rep_count}/{self.target_reps}')

if __name__ == "__main__":
    master_video = r"C:\AIML\assets_master\crunch_master.mp4"
    target_reps = 6
    recognizer = CrunchRecognizer(master_video, target_reps)
    recognizer.process_webcam()


Loading cached master landmarks from CSV...
Final Form Similarity: 97%
Total Time Taken: 24.79 seconds
Total Reps Completed: 6/6
